In [1]:
#  ***** After this step restart the kernel*****

# pip install pandas numpy sqlalchemy matplotlib seaborn kagglehub
# pip install mysql-connector-python sqlalchemy pandas
# pip install mysql-connector-python


Defaulting to user installation because normal site-packages is not writeable


### Uploading Data

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sqlalchemy import create_engine

In [4]:
file_path="all_reviews.csv"
df = pd.read_csv(file_path, low_memory=False)

In [5]:
df.head(2)

,rating,title,status,pros,cons,advice,Recommend,CEO Approval,Business Outlook,Career Opportunities,Compensation and Benefits,Senior Management,Work/Life Balance,Culture & Values,Diversity & Inclusion,firm_link,date,job,index
0,5.0,Good,"Current Employee, more than 10 years",Knowledge gain of complete project,Financial growth and personal growth,NaN,v,o,v,3,3,3,3,3.0,3.0,Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm,"Nov 19, 2022",Manager Design,NaN
1,4.0,Good,"Former Employee, less than 1 year","Good work,good work , flexible, support","Good,work, flexible,good support, good team work",NaN,v,o,o,4,4,4,4,4.0,4.0,Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm,"Jan 29, 2022",Anonymous Employee,NaN


### Data Cleaning & Processing

In [6]:
df=df.drop(columns=['index','advice'],errors='ignore')

In [7]:
df2=df

#### Data Manipulation 

In [8]:
# change columns to boolean format and create new columns
df['date']= pd.to_datetime(df['date'])
df['recommend'] = df['Recommend'].map({'v': True, 'o': False})
df['ceo_approval'] = df['CEO Approval'].map({'v': True, 'o': False})
df['business_outlook'] = df['Business Outlook'].map({'v': True, 'o': False})

In [9]:
df.describe()


,rating,Culture & Values,Diversity & Inclusion,date
count,9.901718e+06,7.655116e+06,3.357607e+06,9901718
mean,3.557778e+00,3.481324e+00,3.841908e+00,2019-04-05 18:43:50.680939520
min,1.000000e+00,1.000000e+00,1.000000e+00,2008-01-25 00:00:00
25%,3.000000e+00,3.000000e+00,3.000000e+00,2016-12-01 00:00:00
50%,4.000000e+00,4.000000e+00,4.000000e+00,2020-06-06 00:00:00
75%,5.000000e+00,5.000000e+00,5.000000e+00,2021-11-16 00:00:00
max,5.000000e+00,5.000000e+00,5.000000e+00,2023-07-26 00:00:00
std,1.249354e+00,1.389978e+00,1.273648e+00,NaN


In [10]:
# Convert the specified columns to float
df['Career Opportunities'] = pd.to_numeric(df['Career Opportunities'], errors='coerce')
df['Senior Management'] = pd.to_numeric(df['Senior Management'], errors='coerce')
df['Work/Life Balance'] = pd.to_numeric(df['Work/Life Balance'], errors='coerce')



# Fill missing numeric values with median
df['rating'].fillna(df['rating'].median(), inplace=True)
df['Career Opportunities'].fillna(df['Career Opportunities'].median(), inplace=True)
df['Senior Management'].fillna(df['Senior Management'].median(), inplace=True)
df['Work/Life Balance'].fillna(df['Work/Life Balance'].median(), inplace=True)
df['Culture & Values'].fillna(df['Culture & Values'].median(), inplace=True)
df['Diversity & Inclusion'].fillna(df['Diversity & Inclusion'].median(), inplace=True)

# Fill missing categorical values with "Unknown"
df['recommend'].fillna("Unknown", inplace=True)
df['ceo_approval'].fillna("Unknown", inplace=True)
df['business_outlook'].fillna("Unknown", inplace=True)

df['date'] = df['date'].dt.normalize()


In [11]:
df.columns

Index(['rating', 'title', 'status', 'pros', 'cons', 'Recommend',
       'CEO Approval', 'Business Outlook', 'Career Opportunities',
       'Compensation and Benefits', 'Senior Management', 'Work/Life Balance',
       'Culture & Values', 'Diversity & Inclusion', 'firm_link', 'date', 'job',
       'recommend', 'ceo_approval', 'business_outlook'],
      dtype='object')

#### Extracting Company name from Links

In [12]:
import re

def extract_company_name(link):
    if pd.isna(link) or link.strip() == "":
        return "Unknown"

    # Case 1: If the link is a relative path (e.g., Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm)
    match = re.search(r'Reviews/([^-/]+(?:-[^-/]+)*)-Reviews', link)
    
    if match:
        company_name = match.group(1).replace("-", " ")  # Replace hyphens with spaces
        return company_name

    # Case 2: If the link is a full URL (e.g., https://www.glassdoor.com/Reviews/Calgary-Flam...)
    match = re.search(r'glassdoor\.com/Reviews/([^-/]+(?:-[^-/]+)*)-Reviews', link)
    
    if match:
        company_name = match.group(1).replace("-", " ")  # Replace hyphens with spaces
        return company_name

    return "Unknown"

# Apply the function
df['company_name'] = df['firm_link'].apply(lambda x: extract_company_name(str(x)))

# Verify the extraction
print(df[['firm_link', 'company_name']].head(10))


                                           firm_link          company_name
0  Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm  Baja Steel and Fence
1  Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm  Baja Steel and Fence
2  Reviews/Baja-Steel-and-Fence-Reviews-E5462645.htm  Baja Steel and Fence
3  https://www.glassdoor.com/Reviews/Calgary-Flam...        Calgary Flames
4  https://www.glassdoor.com/Reviews/Calgary-Flam...        Calgary Flames
5  https://www.glassdoor.com/Reviews/Calgary-Flam...        Calgary Flames
6  https://www.glassdoor.com/Reviews/Calgary-Flam...        Calgary Flames
7  https://www.glassdoor.com/Reviews/Calgary-Flam...        Calgary Flames
8  https://www.glassdoor.com/Reviews/Calgary-Flam...        Calgary Flames
9  https://www.glassdoor.com/Reviews/Calgary-Flam...        Calgary Flames


In [13]:
df.drop(columns=['Recommend', 'CEO Approval', 'Business Outlook', 'firm_link'], inplace=True, errors='ignore')

In [14]:
df['company_name'].fillna('Unknown', inplace=True)
unknown_count = df[df['company_name'] == "Unknown"].shape[0]
print(f"Number of Unknown company names: {unknown_count}")


Number of Unknown company names: 5574


In [15]:
df[df['company_name'] == "Unknown"].head()


,rating,title,status,pros,cons,Career Opportunities,Compensation and Benefits,Senior Management,Work/Life Balance,Culture & Values,Diversity & Inclusion,date,job,recommend,ceo_approval,business_outlook,company_name
14592,4.0,Title,"Current Employee, more than 3 years",Pros include as many training opportunities as...,"Cons include 12 hours shifts, rather than 24 o...",3.0,2,2.0,4.0,5.0,4.0,2016-03-27,Firefighter/Paramedic,True,False,True,Unknown
43825,4.0,"Awesome job, ok company","Current Employee, more than 3 years","The job is wonderful, the company is great, bu...",Absolutely no pay increases our benefits of an...,4.0,2,4.0,3.0,5.0,4.0,2015-06-13,Anonymous Employee,True,False,True,Unknown
94504,4.0,Great Learning Opportunity,Former Intern,This is an amazing experience for any inter. Y...,No pay. Due to this being a government interns...,3.0,NaN,3.0,4.0,4.0,4.0,2017-07-12,Anonymous Intern,True,False,False,Unknown
128619,5.0,"Overall, positive experience with a tremendous...","Former Employee, more than 5 years","Very goal oriented, tightly knit team with pos...",Sometimes difficult balancing between the engi...,4.0,4.0,4.0,4.0,4.0,4.0,2012-09-22,Customer Care Lead,True,False,Unknown,Unknown
128620,4.0,I worked there 2001,"Former Employee, more than 1 year",Great company culture and loved helping the co...,I was not able to move up due to the founders ...,2.0,4.0,3.0,4.0,4.0,4.0,2021-06-17,Vice President of Product Development,True,True,True,Unknown


In [16]:
print(df['company_name'].nunique())
print(df['company_name'].value_counts().head(10))  # Show most common company names


34158
company_name
Amazon                            163396
Tata Consultancy Services         107218
Walmart                           102152
Cognizant Technology Solutions     84171
McDonald s                         76777
Accenture                          69026
Target                             67885
HP Inc                             63787
Starbucks                          55325
Infosys                            53189
Name: count, dtype: int64


#### Eleminating Null Values

In [17]:
print(df.isnull().sum())  # Should be 0 if everything is replaced


rating                             0
title                          39424
status                           171
pros                             188
cons                             234
Career Opportunities               0
Compensation and Benefits    1878091
Senior Management                  0
Work/Life Balance                  0
Culture & Values                   0
Diversity & Inclusion              0
date                             171
job                              171
recommend                          0
ceo_approval                       0
business_outlook                   0
company_name                       0
dtype: int64


In [18]:
df['job'] = df['job'].fillna('Unknown')  # Fill NaN values
df['job'] = df['job'].replace('', 'Unknown')  # Replace empty strings


In [19]:
print(df['job'].isnull().sum())  # Should be 0 if everything is replaced
df['job'].value_counts().head(10)  # Check most common job titles


0


job
 Anonymous Employee                 1704869
                                     682948
 Sales Associate                     155604
 Software Engineer                   147930
 Manager                             127789
 Cashier                              96628
 Customer Service Representative      91980
 Senior Software Engineer             76180
 Associate                            68661
 Assistant Manager                    68181
Name: count, dtype: int64

In [20]:
df['Compensation and Benefits'] = pd.to_numeric(df['Compensation and Benefits'], errors='coerce')
df['Compensation and Benefits'].fillna(df['Compensation and Benefits'].median(), inplace=True)

df['recommend'] = df['recommend'].astype('category')
df['ceo_approval'] = df['ceo_approval'].astype('category')
df['business_outlook'] = df['business_outlook'].astype('category')


In [21]:
# Check for missing (NaN) values in the date column
missing_dates = df[df['date'].isnull()]
print(f"Number of rows with missing date values: {missing_dates.shape[0]}")

# Display the first few rows with missing date values
missing_dates.head(10)


Number of rows with missing date values: 171


,rating,title,status,pros,cons,Career Opportunities,Compensation and Benefits,Senior Management,Work/Life Balance,Culture & Values,Diversity & Inclusion,date,job,recommend,ceo_approval,business_outlook,company_name
43535,4.0,NaN,NaN,NaN,NaN,3.0,3.0,3.0,4.0,4.0,4.0,NaT,Unknown,Unknown,Unknown,Unknown,Tata Group
82740,4.0,NaN,NaN,NaN,NaN,3.0,3.0,3.0,4.0,4.0,4.0,NaT,Unknown,Unknown,Unknown,Unknown,GameStop
105914,4.0,NaN,NaN,NaN,NaN,3.0,3.0,3.0,4.0,4.0,4.0,NaT,Unknown,Unknown,Unknown,Unknown,Chipotle
161539,4.0,NaN,NaN,NaN,NaN,3.0,3.0,3.0,4.0,4.0,4.0,NaT,Unknown,Unknown,Unknown,Unknown,Wendy's
253058,4.0,NaN,NaN,NaN,NaN,3.0,3.0,3.0,4.0,4.0,4.0,NaT,Unknown,Unknown,Unknown,Unknown,Nordstrom
278053,4.0,NaN,NaN,NaN,NaN,3.0,3.0,3.0,4.0,4.0,4.0,NaT,Unknown,Unknown,Unknown,Unknown,Allianz
359887,4.0,NaN,NaN,NaN,NaN,3.0,3.0,3.0,4.0,4.0,4.0,NaT,Unknown,Unknown,Unknown,Unknown,PwC
390211,4.0,NaN,NaN,NaN,NaN,3.0,3.0,3.0,4.0,4.0,4.0,NaT,Unknown,Unknown,Unknown,Unknown,PwC
513361,4.0,NaN,NaN,NaN,NaN,3.0,3.0,3.0,4.0,4.0,4.0,NaT,Unknown,Unknown,Unknown,Unknown,Anika Therapeutics
521227,4.0,NaN,NaN,NaN,NaN,3.0,3.0,3.0,4.0,4.0,4.0,NaT,Unknown,Unknown,Unknown,Unknown,Connection


In [22]:
df = df.dropna(subset=['date'])
print(f"Missing date values after drop: {df['date'].isnull().sum()}")  # Should be 0


Missing date values after drop: 0


In [23]:
df = df.dropna(subset=['pros', 'cons'])

In [24]:
# Count missing values in 'title'
missing_titles = df['title'].isnull().sum()
print(f"Missing values in 'title': {missing_titles}")

# Display rows where 'title' is missing
df_missing_title = df[df['title'].isnull()]
df_missing_title.head(3)  # Show first 10 missing title rows


Missing values in 'title': 39253


,rating,title,status,pros,cons,Career Opportunities,Compensation and Benefits,Senior Management,Work/Life Balance,Culture & Values,Diversity & Inclusion,date,job,recommend,ceo_approval,business_outlook,company_name
88,4.0,NaN,Current Employee,"Family oriented, good management, not much tur...","Less pay, a lot of travel",3.0,3.0,3.0,4.0,4.0,4.0,2022-09-22,Field Marketing Manager,False,False,False,California Casualty
316,3.0,NaN,Former Employee,Good employee benefits and time off,Repetitive work environment long hours,3.0,4.0,3.0,3.0,3.0,3.0,2023-04-29,Sales Support Associate,False,False,False,Canada Life
608,5.0,NaN,"Current Employee, more than 3 years",great place to work at.\r\ngreat benefits\r\nT...,The call center got super busy because of the ...,4.0,5.0,4.0,4.0,4.0,4.0,2022-02-14,Customer Service Representative (Bilingual),False,True,False,Canada Life


In [25]:
df['title'] = df['title'].fillna(df['pros'])  # Use 'pros' column as title
df['title'] = df['title'].fillna(df['cons'])  # Use 'cons' if 'pros' is also missing

# Replace any remaining NaNs with "No Title"
df['title'].fillna("No Title", inplace=True)

In [26]:
df = df.dropna(subset=['title'])


In [27]:
print(df.isnull().sum()) 

rating                       0
title                        0
status                       0
pros                         0
cons                         0
Career Opportunities         0
Compensation and Benefits    0
Senior Management            0
Work/Life Balance            0
Culture & Values             0
Diversity & Inclusion        0
date                         0
job                          0
recommend                    0
ceo_approval                 0
business_outlook             0
company_name                 0
dtype: int64


#### formating data in the columns

In [28]:
# Function to extract employment status
def extract_employment_status(status):
    if "Current Employee" in status:
        return "Current Employee"
    elif "Former Employee" in status:
        return "Former Employee"
    return "Unknown"  # If neither is found

# Apply function to create new column
df['employment_status'] = df['status'].apply(lambda x: extract_employment_status(str(x)))

In [29]:
import re

# Function to extract only the numeric experience years
def extract_experience_years(status):
    match = re.search(r'(\d+)', status)  # Find the first number in the text
    if match:
        return int(match.group(1))  # Convert the matched number to an integer
    return np.nan  # Return NaN if no number is found

# Apply function to create new column
df['experience_years'] = df['status'].apply(lambda x: extract_experience_years(str(x)))


In [30]:
df[['status', 'employment_status', 'experience_years']].head(20)


,status,employment_status,experience_years
0,"Current Employee, more than 10 years",Current Employee,10.0
1,"Former Employee, less than 1 year",Former Employee,1.0
2,"Current Employee, more than 1 year",Current Employee,1.0
3,"Current Employee, more than 1 year",Current Employee,1.0
4,"Current Employee, more than 3 years",Current Employee,3.0
5,"Former Employee, more than 5 years",Former Employee,5.0
6,"Former Employee, less than 1 year",Former Employee,1.0
7,"Former Employee, more than 5 years",Former Employee,5.0
8,"Current Employee, more than 1 year",Current Employee,1.0
9,"Former Employee, more than 1 year",Former Employee,1.0


In [31]:
df['experience_years'].fillna(0,inplace=True)

In [ ]:
# Function to clean text columns
def clean_text(text):
    if pd.isna(text):  # Handle NaN values
        return "Unknown"
    
    text = text.strip()  # Remove leading & trailing spaces
    text = text.lower()  # Convert to lowercase for consistency
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^a-zA-Z0-9.,!? ]', '', text)  # Remove special characters except punctuation
    return text

# cleaning function
df['title'] = df['title'].apply(clean_text)
df['pros'] = df['pros'].apply(clean_text)
df['cons'] = df['cons'].apply(clean_text)


In [ ]:
# Limit title to 100 characters, pros/cons to 300 characters
df['title'] = df['title'].str[:100]
df['pros'] = df['pros'].str[:300]
df['cons'] = df['cons'].str[:300]


In [ ]:
df[['title', 'pros', 'cons']].head(10)


In [ ]:
# Filter rows where all three columns have "Unknown"
unknown_rows = df[(df['title'] == "Unknown") & (df['pros'] == "Unknown") & (df['cons'] == "Unknown")]

# Display sample rows
print("Sample rows where title, pros, and cons are all 'Unknown':")
unknown_rows.head(10)  # Show first 10 rows


In [ ]:
# Count the number of rows where all three columns are "Unknown"
unknown_count = unknown_rows.shape[0]
print(f"Total rows where title, pros, and cons are all 'Unknown': {unknown_count}")


In [ ]:
df.shape

In [ ]:
# Drop rows where all three columns are "Unknown"
df = df[~((df['title'] == "Unknown") & (df['pros'] == "Unknown") & (df['cons'] == "Unknown"))]

print(f"Total rows after cleaning: {df.shape[0]}")


In [ ]:
# Drop the 'status' column as it is no longer needed
df.drop(columns=['status'], inplace=True, errors='ignore')

# Verify that the column has been removed
print(df.info())

In [ ]:
# Rearranging column order
new_column_order = [
    "date", "company_name", "job", "employment_status", "experience_years",  # Employee Info
    "title", "rating", "pros", "cons",  # Review Details
    "Career Opportunities", "Compensation and Benefits", "Senior Management",  # Ratings
    "Work/Life Balance", "Culture & Values", "Diversity & Inclusion",  # More Ratings
    "recommend", "ceo_approval", "business_outlook"  # Recommendation & Outlook
]

# Reorder the DataFrame columns
df = df[new_column_order]

# Verify the new column order
df.head(3)


In [ ]:
# Dictionary to rename columns (Edit as needed)
column_mapping = {
    "Career Opportunities": "career_opportunities",
    "Compensation and Benefits": "compensation_benefits",
    "Senior Management": "senior_management",
    "Work/Life Balance": "work_life_balance",
    "Culture & Values": "culture_values",
    "Diversity & Inclusion": "diversity_inclusion",
    "recommend": "recommendation",
    "ceo_approval": "ceo_approval",
    "business_outlook": "business_outlook",
    "company_name": "company",
    "employment_status": "employment_status",
    "experience_years": "years_experience",
    "job": "job_title",
    "title": "review_title",
    "pros": "positives",
    "cons": "negatives",
    "date": "review_date",
    "rating": "overall_rating"
}


# Apply renaming only for existing columns
df.rename(columns=column_mapping, inplace=True)

# Display updated column names
print("✅ Column names updated successfully!")
print(df.columns)
df.head(2)

In [ ]:
# Replace blank job titles with "Unknown"
df['job_title'] = df['job_title'].apply(lambda x: "Unknown" if str(x).strip() == "" else x)

# Verify the changes
print(f"Total missing job titles: {df['job_title'].isnull().sum()}")
print(df['job_title'].value_counts().head(10))  # Check most common job titles


### EDA Exploratory Data Analysis

In [ ]:
df.head(2)

### Sending the data into SQL using SQLAlchemy

In [ ]:

# import mysql.connector

# host = "10.48.122.213"  # Use the IP from Step 2
# port = 3307
# user = "root"
# password = "lokesh0204@B"
# database = "glassdoor"

# try:
#     conn = mysql.connector.connect(
#         host=host,
#         port=port,
#         user=user,
#         password=password,
#         database=database
#     )
#     cursor = conn.cursor()
#     print("✅ Connected to MySQL successfully!")
# except mysql.connector.Error as err:
#     print(f"❌ Connection failed: {err}")


In [ ]:
# cursor.execute("SELECT COUNT(*) FROM glassdoor_reviews;")
# record_count = cursor.fetchone()[0]
# print(f"Total records in glassdoor_reviews: {record_count}")

# cursor.execute("SELECT * FROM glassdoor_reviews LIMIT 5;")
# sample_rows = cursor.fetchall()
# for row in sample_rows:
#     print(row)


In [ ]:
import sqlalchemy
import pandas as pd

print("✅ SQLAlchemy and Pandas are installed correctly!")


In [ ]:
# data_tuples = df.to_records(index=False).tolist()  # Convert to list of tuples


In [ ]:
# import mysql.connector

# # Reconnect to MySQL
# conn = mysql.connector.connect(
#     host="10.29.177.110",
#     port=3307,
#     user="root",
#     password="lokesh0204@B",
#     database="glassdoor_db"
# )
# cursor = conn.cursor()

# # Define batch size
# batch_size = 10000  

# # Generate INSERT SQL query
# columns = ",".join([f"`{col}`" for col in df.columns]) 
# values_placeholder = ",".join(["%s"] * len(df.columns))
# sql_query = f"INSERT INTO glassdoor_reviews ({columns}) VALUES ({values_placeholder})"

# # Insert data in batches
# for i in range(0, len(data_tuples), batch_size):
#     batch = data_tuples[i:i + batch_size]
#     cursor.executemany(sql_query, batch)
#     conn.commit()
#     print(f" Inserted {i + batch_size} rows successfully...")

# # Close connection
# cursor.close()
# conn.close()
# print(" Upload Completed!")


In [ ]:
# import mysql.connector

# host = "10.48.122.213"  # ✅ Corrected IP
# port = 3307
# user = "root"
# password = "lokesh0204@B"
# database = "glassdoor"

# try:
#     conn = mysql.connector.connect(
#         host=host,
#         port=port,
#         user=user,
#         password=password,
#         database=database
#     )
#     cursor = conn.cursor()
#     cursor.execute("SHOW TABLES;")
#     tables = cursor.fetchall()
#     print("✅ Connected to MySQL! Tables in database:", tables)
#     conn.close()
# except mysql.connector.Error as err:
#     print(f"❌ Connection failed: {err}")
